# FFWIZARD.COM --- QB and DEF Combos

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import StringIO
from sklearn import preprocessing
from pandas import ExcelWriter
from datetime import datetime

### Load NFL Schedule

In [2]:
# load nfl schedule data (manually entered)
def load_schedule():
    nfl_sched = pd.read_csv('2019_NFL_Schedule.csv')
    return nfl_sched

### Defense Strength Data

In [3]:
def calc_defense_strength(bye_adjustor):

    # load nfl defense strength data (manually entered)
    def_strength_data = pd.read_csv('2019_Defense_Strength.csv')

    #remove 'source' row
    def_strength = def_strength_data.iloc[:-1, :]

    # make weights based on quality of rankings
    def_rank_weights = pd.Series([0.20, 0.225, 0.225, 0.10, 0.10, 0.075, 0.075])
    
    # create weighted defense rankings array based on weights
    num_teams = len(def_strength)
    weighted_def_rank = np.empty(num_teams, dtype=float)
    
    for i in range(0, num_teams):
        weighted_def_rank[i] = def_rank_weights[0] * float(def_strength.iloc[i, 1]) + def_rank_weights[1] * float(def_strength.iloc[i, 2]) + def_rank_weights[2] * float(def_strength.iloc[i, 3]) + def_rank_weights[3] * float(def_strength.iloc[i, 4])
    weighted_def_rank_df = pd.Series(weighted_def_rank)

    # scale weighted defense rankings to have mean of 0 and std dev of 1
    def_rank_scaled = pd.Series(preprocessing.scale(weighted_def_rank_df))

    # create table with team and score info
    defense_score_no_bye = pd.concat([def_strength.loc[:, "Team"], weighted_def_rank_df, def_rank_scaled], axis=1)
    col_names = ["Team", "Def_Score", "Def_Score_Scaled"]
    defense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    defense_score = pd.concat([defense_score_no_bye, bye_array])

    defense_score = defense_score.reset_index(drop=True)
    
    return defense_score

### Matchup Scores by Week

In [4]:
def matchup_scores_by_week(schedule, score, last_week):

    schedule_score = schedule.copy()

    # replace schedule with actual matchup scores
    for i in range(0, schedule.shape[0]):
        for j in range(1, schedule.shape[1]):
            for k in range(0, len(score)):
                if score.loc[k, "Team"] == schedule.iloc[i, j]:
                    schedule_score.iloc[i, j] = score.iloc[k, 2]

    custom_sched_score = schedule_score.iloc[:, :(last_week+1)]

    return custom_sched_score

### Inputs and Run Functions

In [5]:
bye_adjustor = -2.5
schedule = load_schedule()
score = calc_defense_strength(bye_adjustor)
target_team_list = range(0, 32)
last_week_list = [17]


In [6]:
dummy = matchup_scores_by_week(schedule, score, last_week_list[0])

In [7]:
dummy

,Team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,ARI,0.428042,-1.12205,0.81037,0.282591,1.54594,0.660764,1.47945,-0.457132,0.901797,1.73295,0.901797,-2.5,-0.856084,-0.444665,-0.494534,0.282591,-0.856084
1,ATL,-1.53763,-0.157918,-0.16623,-0.407263,-0.673231,0.897641,-0.856084,0.282591,-2.5,-0.457132,0.81037,1.73295,-0.457132,0.81037,0.901797,-1.57087,1.73295
2,BAL,1.50854,0.897641,0.544403,-0.494534,-0.444665,1.54594,0.282591,-2.5,-0.748034,1.54594,-0.673231,-0.856084,0.901797,-0.386484,0.353238,-0.494534,-0.444665
3,BUF,0.353238,1.47945,1.54594,-0.748034,-0.407263,-2.5,1.50854,-0.157918,0.340771,-0.494534,1.50854,-0.669075,-0.685698,-1.12205,-0.444665,-0.748034,0.353238
4,CAR,-0.856084,1.73295,0.897641,-0.673231,-1.57087,1.73295,-2.5,0.901797,-0.407263,0.261812,0.660764,-0.457132,0.340771,0.660764,0.282591,-0.16623,-0.457132
5,CHI,0.261812,-0.669075,0.340771,-1.53763,1.84515,-2.5,-0.457132,-1.28413,-0.157918,0.428042,-0.856084,1.47945,0.428042,-0.685698,0.261812,0.544403,-1.53763
6,CIN,0.282591,0.901797,-0.386484,-0.444665,0.897641,-1.12205,-1.57087,-0.856084,-2.5,-1.12205,1.84515,-0.444665,0.353238,-0.494534,-0.748034,1.50854,-0.494534
7,CLE,-0.407263,0.353238,-0.856084,-1.12205,0.901797,0.282591,-2.5,-0.748034,-0.669075,-0.386484,-0.444665,1.50854,-0.444665,1.54594,0.897641,-1.12205,1.54594
8,DAL,1.47945,0.340771,1.50854,-0.457132,0.261812,0.353238,-0.157918,-2.5,1.47945,-1.53763,0.428042,-0.748034,-0.386484,-1.93242,-0.856084,-0.157918,0.340771
9,DEN,1.84515,-1.93242,0.261812,-1.57087,-1.28413,-0.407263,0.544403,-0.16623,-0.494534,-2.5,-1.53763,-0.386484,-1.28413,-0.673231,0.544403,0.428042,1.84515


In [8]:
target_team = 1
target_matchups = dummy.iloc[target_team, :]

In [9]:
target_matchups

Team         ATL
1       -1.53763
2      -0.157918
3       -0.16623
4      -0.407263
5      -0.673231
6       0.897641
7      -0.856084
8       0.282591
9           -2.5
10     -0.457132
11       0.81037
12       1.73295
13     -0.457132
14       0.81037
15      0.901797
16      -1.57087
17       1.73295
Name: 1, dtype: object

In [10]:
def combo_score(target_team, custom_sched_score, last_week):

    target_team_name = custom_sched_score.loc[target_team, "Team"]
    col_label = target_team_name + "_" + str(last_week)

    target_matchups = custom_sched_score.iloc[target_team, :]

    combos_calc = custom_sched_score.copy()

    sum_combos_score = []

    for i in range(0, custom_sched_score.shape[0]):
        for j in range(1, custom_sched_score.shape[1]):
            if target_matchups[j] > custom_sched_score.iloc[i, j]:
                combos_calc.iloc[i, j] = 0
            else:
                combos_calc.iloc[i, j] = custom_sched_score.iloc[i, j] - target_matchups[j]

        sum_combos_score.append(combos_calc.iloc[i, 1:].sum())

    sum_combos_score_df = pd.DataFrame({col_label: sum_combos_score})

    targ_team_combo = pd.concat([custom_sched_score.loc[:, "Team"], sum_combos_score_df, custom_sched_score.iloc[:, 1:]], axis=1)

    targ_team_combo_sorted = targ_team_combo.sort_values(by=[col_label], ascending=False)
    
    return targ_team_combo_sorted, col_label

### Inputs

In [11]:
bye_adjustor = -2.5
schedule = load_schedule()
def_score = calc_defense_strength(bye_adjustor)

target_team_list = range(0, 32)
last_week_list = [16]

### Make Excel File with Defense Matchup Strengths

In [12]:
# make writer object for Excel file
writer = pd.ExcelWriter('./combos/qb_combos.xlsx', engine='xlsxwriter')

# iterate through every input permutation
for i in range(0, len(last_week_list)):
    
    for j in range(0, len(target_team_list)):
        
        
        custom_sched_score = matchup_scores_by_week(schedule, def_score, last_week_list[i])
        targ_team_combo_sorted, col_label = combo_score(target_team_list[j], custom_sched_score, last_week_list[i])
        
        targ_team_combo_sorted.to_excel(writer, sheet_name=col_label)
        
        print col_label

writer.save()

ARI_16
ATL_16
BAL_16
BUF_16
CAR_16
CHI_16
CIN_16
CLE_16
DAL_16
DEN_16
DET_16
GB_16
HOU_16
IND_16
JAX_16
KC_16
LAC_16
LAR_16
MIA_16
MIN_16
NE_16
NO_16
NYG_16
NYJ_16
OAK_16
PHI_16
PIT_16
SEA_16
SF_16
TB_16
TEN_16
WAS_16
